In [12]:
import numpy as np 
from scipy.stats import norm

class OptionsMath():
    
    """
    Preface:
        The classic Black Scholes formula as implemented below assumes:
            1) The amount of time that is used for the determining the distribution of the underlying value 
            at expiry is the same amount of time that should be used for discounting the forward price of 
            the option.  Frequently these two amounts of time are different as options rarely settle at expiry
            but instead settle at expiry plus a few days.
            2) The interest rate used to project the underlying value at expiry based on the underlying spot 
            value is the same as the interest rate that should be used for discounting the forward price of
            the option.  In crytpo markets in 2025, for example, these two rates are very different.
        To overcome these shortcomings, the following implementation is recommended:
            1) Set the interest rate in the Black Scholes formula to zero.
            2) Calculate the underlying value at expiry prior to calling the formula.  Enter the underlying
            value at epxiry as the undelrying_value variable# Assistant
            3) Calculate the amount of time to expiration of the option prior to calling the formual.  Set
            the time variable equal to this amount.
            4) Discount the formulas's resulting forward price of the option after using the formula below
            using the Time_Value_Money Class and choices for rate and time that are specific to the discounting
            situation (time to payment rather than time to expiry and rate for discounting rather than forward
            rate of underlying).
        The term "value" is used rather than "price" to emphasize that formulas below can be used for rates, too
    
    Parameters:
        c_or_p (string): acceptable names for 'call' or 'put' (see c_or_p function below)
        u (float): see note above, value of underlying
        t (float): see note above, fraction of a year (use Daycount_Math prior to calling functions below)
        k (float): strike value expressed in same units as underlying_value
        v (float): volatility expressed as a percentage of underlying value (should be the same value regardless of using 
            forward or spot underlying price)
        r (float): see note above, continuously compounded rate used to (i) discount forward option 
            value to present value and (ii) create underlying value at expiry (if spot underlying value is used
            as the input) 
        d1 (float): the amount equal to the d1 term in the Black Scholes formula
        d2 (float): the amount equal to the d2 term in the Black Scholes formula
    """
        
    def __init__(self):
        # constructor 
        pass


    # -----------------------
    # helpers
    # -----------------------
    def pv_factor(self, t=t, r=r):
        # Compute time value of money - eventually replace with function from TVM Class
        return np.exp(-r * t) #if interest_rate == 0 then pv_factor will equal one

        
    def to_arrays(self, *args, d_type=float):
        # Convert all inputs to NumPy arrays of dtype=d_type.
        # Raises ValueError if any array contains NaN values.
        # Returns a tuple of arrays in the same order as the inputs.
        arrays = [np.asarray(arg, dtype=d_type) for arg in args]
            
        # Only check NaN for numeric dtypes
        for i, arr in enumerate(arrays):
            if np.issubdtype(arr.dtype, np.number) and np.any(np.isnan(arr)):
                raise ValueError(f"Array {i+1} contains NaN values")
                    
        return tuple(arrays)

    
    def c_or_p(self, c_or_p, array_to_mimic):        
        # Convert array_to_mimic so we can broadcast to its shape if needed
        array_to_mimic, = self.to_arrays(array_to_mimic)
        
        # If scalar indicator, broadcast to array_to_mimic's shape
        if np.isscalar(c_or_p):
            c_or_p = np.full_like(array_to_mimic, c_or_p, dtype=object)

        # Convert to object array and uppercase elementwise
        c_or_p, = self.to_arrays(c_or_p, d_type=object)
        c_or_p = np.char.upper(c_or_p.astype(str))

        call_names = ['C', 'CALL']
        put_names  = ['P', 'PUT']
        valid_names = call_names + put_names
        
        # Validate input
        if not np.all(np.isin(c_or_p, valid_names)):
            raise ValueError("c_or_p must contain only values associated with 'call' or 'put'")

        return np.where(np.isin(c_or_p, call_names), 'CALL', 'PUT')
            
                
    # -----------------------
    # d-terms
    # -----------------------
    def bs_d0(self, u=np.nan, k=np.nan, t=np.nan, v=np.nan, r=0, **kwargs):
        # Convert inputs to NumPy arrays for elementwise math
        u, k, t, v, r = self.to_arrays(u, k, t, v, r)

        # Prevent division by zero or invalid sqrt(time)
        t = np.maximum(t, 1e-12)
        v = np.maximum(v, 1e-12)

        # Core math
        vol_sqrt_time = v * np.sqrt(t)     
        log_moneyness = np.log(u / k)
        growth_rate = r * t # if interest_rate == 0 then growth_rate will equal one 
        
        return (log_moneyness + growth_rate) / vol_sqrt_time
        

    def bs_d1_given_d0(self, d0=np.nan, t=np.nan, v=np.nan, **kwargs):
        # Convert inputs to NumPy arrays for elementwise math
        d0, v, t = self.to_arrays(d0, v, t)
                                 
        return d0 + (v * np.sqrt(t) / 2.0)


    def bs_d2_given_d0(self, d0=np.nan, t=np.nan, v=np.nan, **kwargs):
        # Convert inputs to NumPy arrays for elementwise math
        d0, v, t = self.to_arrays(d0, v, t)
                                 
        return d0 - (v * np.sqrt(t) / 2.0)

    
    def bs_d1(self, u=np.nan, k=np.nan, t=np.nan, v=np.nan, r=0, **kwargs):
        d0 = self.bs_d0(u, k, t, v, r)
        return self.bs_d1_given_d0(d0, t, v)

    
    def bs_d2(self, u=np.nan, k=np.nan, t=np.nan, v=np.nan, r=0, **kwargs):
        d0 = self.bs_d0(u, k, t, v, r)
        return self.bs_d2_given_d0(d0, t, v)

    
    # -----------------------
    # pricing formulas
    # -----------------------         
    def bs_value_euro_given_d1_d2(self, d1=np.nan, d2=np.nan, c_or_p='', 
                                  u=np.nan, k=np.nan, t=np.nan, r=0, **kwargs):                      
        # Convert all inputs to NumPy arrays for broadcasting
        d1, d2, u, k, t, r = self.to_arrays(d1, d2, u, k, t, r)

        # Normalize & validate c_or_p (allow scalar or array)
        c_or_p = self.c_or_p(c_or_p, u)
        put_call_scalar = np.where(c_or_p == 'CALL', 1, -1)
        
        # Compute N(d1) and N(d2)
        nd1 = norm.cdf(d1 * put_call_scalar)
        nd2 = norm.cdf(d2 * put_call_scalar)
        
        # Compute final option price components
        underlying_product =  put_call_scalar * nd1 * u 
        strike_product     = -put_call_scalar * nd2 * (k * self.pv_factor(t=t, r=r))
        
        return underlying_product + strike_product                      
                      
                     
    def bs_value_euro(self, c_or_p='', u=np.nan, k=np.nan, t=np.nan, v=np.nan, r=0,  **kwargs):
        # Compute d1 and d2
        d0 = self.bs_d0(u, k, t, v, r)
        d1 = self.bs_d1_given_d0(d0, t, v)
        d2 = self.bs_d2_given_d0(d0, t, v)
        
        return self.bs_value_euro_given_d1_d2(d1=d1, d2=d2, c_or_p=c_or_p, u=u, k=k, t=t, r=r)

        
    # -----------------------
    # greeks (first order)
    # -----------------------
    def bs_delta_d1(self, d1=np.nan, c_or_p='', **kwargs):        
        # Convert all inputs to NumPy arrays for broadcasting
        d1, = self.to_arrays(d1)

        # Normalize & validate c_or_p (allow scalar or array)
        c_or_p = self.c_or_p(c_or_p, d1)
        
        # Compute deltas
        call_delta = norm.cdf(d1)
        put_delta  = call_delta - 1  # N(d1) − 1 = −N(−d1)

        return np.where(c_or_p == 'CALL', call_delta, put_delta)

    
    def bs_delta(self, c_or_p='', u=np.nan, k=np.nan, t=np.nan, v=np.nan, r=0,  **kwargs):       
        # Compute d1
        d1 = self.bs_d1(u, k, t, v, r)
        
        # Compute delta
        return self.bs_delta_d1(d1=d1, c_or_p=c_or_p)
        

    def bs_vega_d1(self, d1=np.nan, u=np.nan, t=np.nan, **kwargs):        
        # Convert all inputs to NumPy arrays for broadcasting
        d1, u, t = self.to_arrays(d1, u, t)

        # Compute vega
        return u * np.sqrt(t) * norm.pdf(d1)  
        
        
    def bs_vega(self, u=np.nan, k=np.nan, t=np.nan, v=np.nan, r=0,  **kwargs):    
        # Compute d1
        d1 = self.bs_d1(u, k, t, v, r)
        
        # Compute vega
        return self.bs_vega_d1(d1, u, t) 

    
    def bs_theta_given_d1_d2(d1=d1, d2=d2, c_or_p=c_or_p, u=u, v=v, t=t, r=r, k=k):
        # Convert all inputs to NumPy arrays for broadcasting    
        d1, d2, u, v, t, r, k = self.to_arrays(d1, d2, u, v, t, r, k)
        
        # Normalize & validate c_or_p (allow scalar or array)
        c_or_p = self.c_or_p(c_or_p, u)
        put_call_scalar = np.where(c_or_p == 'CALL', 1, -1)
        
        # Compute theta terms
        term1 = -1 * u * v * norm.pdf(d1) / (2 * np.sqrt(t))
        term2 = -1 * r * k * norm.cdf(d2 * put_call_scalar) * self.pv_factor(t=t, r=r) * put_call_scalar 
        
        return term1 + term2


    def bs_theta():
        pass


    def bs_rho_given_d2(d2=d2, c_or_p=c_or_p, t=t, r=r, k=k):
        # Convert all inputs to NumPy arrays for broadcasting    
        d2, t, r, k = self.to_arrays(d2, t, r, k)
        
        # Normalize & validate c_or_p (allow scalar or array)
        c_or_p = self.c_or_p(c_or_p, u)
        put_call_scalar = np.where(c_or_p == 'CALL', 1, -1)
        
        # Compute theta terms
        term1 = -1 * u * v * norm.pdf(d1) / (2 * np.sqrt(t))
        term2 = -1 * r * k * norm.cdf(d2 * put_call_scalar) * self.pv_factor(t=t, r=r) * put_call_scalar 
        
        return k * t * norm.cdf(d2 * put_call_scalar) * self.pv_factor(t=t, r=r) * put_call_scalar

    
    def bs_rho():
        pass
























    # -----------------------
    # greeks (second order)
    # -----------------------
    def bs_gamma_d1(self, d1=np.nan, u=np.nan, t=np.nan, v=np.nan):
        # Convert all inputs to NumPy arrays for broadcasting
        d1, u, t, v = self.to_arrays(d1, u, t, v)
        
        # Compute gamma parts
        numerator   = norm.pdf(d1)
        denominator = u * v * np.sqrt(t)

        return numerator / denominator        

    
    def bs_gamma(self, u=np.nan, k=np.nan, t=np.nan, v=np.nan, r=0):        
        # Convert all inputs to NumPy arrays for broadcasting
        u, k, t, v, r = self.to_arrays(u, k, t, v, r)

        # Compute d1 and d2
        d1 = self.bs_d1(u, k, t, v, r)   
        
        return self.bs_gamma_d1(d1, u, t, v)

        
    def bs_volga_d1():
        pass
        #pending
        
        
    def bs_volga():
        pass
        #pending


    # -----------------------
    # greeks (third order)
    # -----------------------
    def bs_speed_d1():
        pass
        #pending

    
    def bs_speed():
        pass
        #pending

    
    def bs_ultima_d1():
        pass
        #pending  


    def bs_ultima():
        pass
        #pending        


    # -----------------------
    # greeks (cross)
    # -----------------------
    def bs_vanna_d1():
        pass
        #pending


    def bs_vanna():
        pass
        #pending


    def bs_charm_d1():
        pass
        #pending
        
    
    def bs_charm():
        pass
        #pending


    def bs_veta_d1():
        pass
        #pending


    def bs_veta():
        pass
        #pending


    def bs_zomma_d1():
        pass
        #pending
        
    
    def bs_zomma():
        pass
        #pending


    def bs_color_d1():
        pass
        #pending


    def bs_color():
        pass
        #pending


    # -----------------------
    # intrinsic value; extrinsic/time value = option value - intrinsic value
    # -----------------------# 
    def intrinsic_value(self, u=u, pv_k=pv_k):
        # Calcs intrinsic values for options
        # Subtract intrinsic values from options prices to get time values
        # Calc the present value of the strike values prior to calling this function
        
        u, pv_k = self.to_arrays(u, pv_k)
        
        call_intrinsic = np.maximum(u - pv_k, 0.0)
        put_intrinsic  = np.maximum(pv_k - u, 0.0)
        
        return call_intrinsic, put_intrinsic
    